# Introduction
**Statement:- We are supposed to produce new content like text generation Using LLM's**   
We will be using GPT-2 model to solve this problem and create more contents out of it.


**First We will read the dataset and extract all the information out of it in a variable, then we will create our mode to train it using the above data, after succesfully creating and saving the model we will test it on certain sets of questions recieved and afterour model is ready. We will need to deploy it, so for that very purpose we will use pickle for that.**

In [2]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.9 MB/s eta 0:00:00


# Loading and reading the dataset

In [3]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    # Open the PDF file in binary mode
    with open(pdf_path, 'rb') as file:
        # Create a PDF file reader object
        pdf_reader = PyPDF2.PdfReader(file)
        
        # Initialize an empty string to store the extracted text
        text = ""
        
        # Loop through each page in the PDF
        for page_num in range(len(pdf_reader.pages)):
            # Extract text from the current page
            page = pdf_reader.pages[page_num]
            text += page.extract_text()
        
        return text

# Path to the PDF file
pdf_path = "/kaggle/input/new-data/48lawsofpower.pdf"

# Extract text from the PDF
extracted_text = extract_text_from_pdf(pdf_path)

In [4]:
import re
extracted_text.split('\n')
extracted_text1 = re.sub('[^A-Za-z]',' ',extracted_text)
extracted_text1.strip()
extracted_text1 = re.sub(r"\s+", " ", extracted_text1.lower())


# Training Of Data 

In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset

# Load pre-trained GPT-2 model and tokenizer
model_name = "gpt2-medium"  # Choose the desired model size
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

training_text = []
training_text.append(extracted_text1)
# Add a padding token to the tokenizer

tokenizer.add_special_tokens({'pad_token': '[PAD]'})

# Tokenize the story data
tokenized_data = tokenizer(training_text, return_tensors="pt", padding=True, truncation=True)

# Create a dataset from the tokenized data
dataset = Dataset.from_dict(tokenized_data)

# Define the training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=8,
    save_steps=10_000,
    save_total_limit=2,
)

# Define the trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
    train_dataset=dataset,
)

# Train the model
trainer.train()
trainer.save_model("./gpt2-finetuned")
tokenizer.save_pretrained("./my_tokenizer")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

...

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss


('./my_tokenizer/tokenizer_config.json',
 './my_tokenizer/special_tokens_map.json',
 './my_tokenizer/vocab.json',
 './my_tokenizer/merges.txt',
 './my_tokenizer/added_tokens.json')

In [8]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer,GenerationConfig

def generate_response(question, model, tokenizer, max_length=100):
    # Tokenize the input question
    
    model.config.pad_token_id = tokenizer.eos_token_id
    input_ids = tokenizer.encode(question, return_tensors="pt")
    attention_mask = torch.ones_like(input_ids)
    

    # Generate response from the model
    generation_config = GenerationConfig(
    max_length=50,
    temperature=1.2,
    num_return_sequences=1
    )

    output = model.generate(input_ids,max_length=generation_config.max_length, temperature=generation_config.temperature)

    # Decode the generated response
    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_response

# Load the fine-tuned GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/kaggle/working/gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("/kaggle/working/my_tokenizer")

# Example questions
questions = [
    'Can you give me an example from history where the enemy was crushed totally from the book?',
    'Whats the point of making myself less accessible?',
    'Can you tell me the story of Queen Elizabeth I from this 48 laws of power book?'
    
]

# Generate responses to each question
for question in questions:
    response = generate_response(question, model, tokenizer)
    print(f"Question: {question}")
    print(f"Answer: {response}\n")


Question: Can you give me an example from history where the enemy was crushed totally from the book?
Answer: Can you give me an example from history where the enemy was crushed totally from the book?

I would say the French in the mid-nineteenth century were able to defeat the Ottoman Empire by throwing away their secular laws and replacing them with a

Question: Whats the point of making myself less accessible?
Answer: Whats the point of making myself less accessible?

I want to make myself more accessible to people who want to learn more about me and my work. I want to make it easier for people to learn more about me and my work by giving

Question: Can you tell me the story of Queen Elizabeth I from this 48 laws of power book?
Answer: Can you tell me the story of Queen Elizabeth I from this 48 laws of power book?

The laws of power by robert greene penguin books hardcover edition isbn paperback edition isbn pages wisdom in a nutshellthe laws of power



Note:- Here as we increase our epochs and max outputs will show less repeated sentences and more variations till saturation, And  more repeated sentences are seen in case of less epochs.

# Loading the model for deploying

In [7]:
import pickle
import torch
import pickle
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Fine-tuned GPT-2 model and tokenizer
model = GPT2LMHeadModel.from_pretrained("/kaggle/working/gpt2-finetuned")
tokenizer = GPT2Tokenizer.from_pretrained("/kaggle/working/my_tokenizer")

# Save model and tokenizer using pickle
with open("fine_tuned_model.pkl", "wb") as f:
    pickle.dump((model, tokenizer), f)

# Load model and tokenizer from pickle file
with open("fine_tuned_model.pkl", "rb") as f:
    loaded_model, loaded_tokenizer = pickle.load(f)


def generate_response(question, model, tokenizer, max_length=100):
    # Tokenize the input question
    
    model.config.pad_token_id = tokenizer.eos_token_id
    input_ids = tokenizer.encode(question, return_tensors="pt")
    attention_mask = torch.ones_like(input_ids)
    

    # Generate response from the model
    generation_config = GenerationConfig(
    max_length= max_length,
    temperature=0.7,
    num_return_sequences=1
    )

    output = model.generate(input_ids,max_length=generation_config.max_length, temperature=generation_config.temperature)

    # Decode the generated response
    generated_response = tokenizer.decode(output[0], skip_special_tokens=True)

    return generated_response

    
questions = [
    'Can you give me an example from history where the enemy was crushed totally from the book?',
    'Whats the point of making myself less accessible?',
    'Can you tell me the story of Queen Elizabeth I from this 48 laws of power book?'
    
]
from transformers import GenerationConfig
for question in questions:
    response = generate_response(question,loaded_model, loaded_tokenizer, max_length=500)
    print(f"Question: {question}")
    print(f"Answer: {response}\n")


/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:389: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


Question: Can you give me an example from history where the enemy was crushed totally from the book?
Answer: Can you give me an example from history where the enemy was crushed totally from the book?

I would say the French in the mid-nineteenth century were able to defeat the Ottoman Empire by throwing away their secular laws and replacing them with a more liberal one. They then turned the tide of the war by throwing in with the Catholics and the Protestants.

What do you think of the idea that the Koran is the word of God?

I think it is a very clever way to get people to believe in a lie. The Koran is not a book of laws written by an omniscient being. It is a compilation of laws from many different cultures and different times. It is not a book of laws by a single prophet. It is a compilation of laws by many different people over a very long period of time.

The fact is that the laws of power are not written in stone. They are constantly evolving and changing over time. The laws of 

**Thank You**